In [1]:
from pyspark import SparkContext 

### SparkContext

In [2]:
sc = SparkContext()

In [3]:
sc.getConf().getAll()

[('spark.app.id', 'local-1702817747313'),
 ('spark.app.startTime', '1702817746208'),
 ('spark.app.submitTime', '1702817746074'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.host', '3339e239d7ca'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.se

In [28]:
sc.stop()

In [29]:
sc = SparkContext(appName='SC Demo',master='local[*]').getOrCreate()

In [4]:
sc.getConf().getAll()

[('spark.driver.port', '45625'),
 ('spark.app.name', 'SC Demo'),
 ('spark.app.startTime', '1702672078185'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1702672078283'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.u

In [15]:
rdd1 = sc.parallelize([1,2,3,4,5,6,1,2,3,4,5,6,1,2,3,4,5,6,1,2,3,4,5,6,1,2,3,4,5,6],9)

In [16]:
rdd1.getNumPartitions()

9

In [18]:
# rdd1.collect()

In [19]:
rdd1.glom().collect()

[[1, 2, 3],
 [4, 5, 6],
 [1, 2, 3],
 [4, 5, 6],
 [1, 2, 3],
 [4, 5, 6],
 [1, 2, 3],
 [4, 5, 6],
 [1, 2, 3, 4, 5, 6]]

In [14]:
# rdd1.saveAsTextFile('sample.txt')

In [22]:
rdd1.repartition(6).glom().collect()

[[4, 5, 6, 1, 2, 3, 4, 5, 6],
 [],
 [1, 2, 3, 4, 5, 6],
 [1, 2, 3, 1, 2, 3],
 [4, 5, 6],
 [1, 2, 3, 4, 5, 6]]

In [23]:
rdd1.count()

30

In [24]:
rdd1.first()

1

In [29]:
rdd1.min()

1

In [30]:
rdd1.max()

6

In [32]:
rdd1.take(15)

[1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3]

In [33]:
rdd1.takeOrdered(15)

[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3]

### Transformations

In [62]:
rdd1 = sc.textFile('economic-survey-of-manufacturing-sept-2023-csv.csv')

rdd1 = rdd1.map(lambda x:x.split(','))

rdd1 = rdd1.filter(lambda x: x[4]!='STATUS')

rdd1 = rdd1.map(lambda x: (x[4],x[2]))

rdd1 = rdd1.map(lambda x: (x[0],'0') if x[1]=='' else x)

rdd1.reduceByKey(lambda x,y: max(float(x),float(y))).collect()

[('F', 35749.295), ('R', 34108.332), ('C', 0.0)]

In [63]:
rdd1.reduceByKey(lambda x,y: min(float(x),float(y))).collect()

[('F', -21.9), ('R', -26.0), ('C', 0.0)]

In [64]:
rdd1.countByKey()

defaultdict(int, {'F': 13455, 'C': 1649, 'R': 13534})

In [69]:
rdd1.keys().collect()[:10]

['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']

In [70]:
rdd1.values().collect()[:10]

['2493.37',
 '3209.2',
 '2780.17',
 '1933.01',
 '2692.92',
 '3079.61',
 '2668.49',
 '1812.22',
 '2916.07',
 '3119.33']

In [81]:
rdd2 = sc.textFile('economic-survey-of-manufacturing-sept-2023-csv.csv')

rdd2 = rdd2.map(lambda x:x.split(','))

rdd2 = rdd2.filter(lambda x: x[4]!='STATUS')

In [82]:
rdd2.partitionBy(4).saveAsTextFile('economic-test.txt')

Py4JJavaError: An error occurred while calling o918.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/jovyan/work/economic-test.txt already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:299)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:407)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctio

In [83]:
rdd2.toDF()

PySparkRuntimeError: [CALL_BEFORE_INITIALIZE] Not supported to call `RDD.toDF` before initialize SparkSession.

In [84]:
sc.stop()